In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# carrega dados
dfDataChampionship = pd.read_parquet("./dados/dados_brasileirao_2003-2023.parquet")

# ordena do mais antigo para mais recente
dfDataChampionship = dfDataChampionship.sort_values("Data", ignore_index=True)

# ajusta dados para String()
dfDataChampionship["Rodada"]           = dfDataChampionship["Rodada"].map(str)
# dfDataChampionship["Mandante_Placar"]  = dfDataChampionship["Mandante_Placar"].map(str)
# dfDataChampionship["Visitante_Placar"] = dfDataChampionship["Visitante_Placar"].map(str)

# mostra dados
dfDataChampionship.head(3)

In [ ]:
# filtra para apenas o Brasileirao 2022
filter1  =  dfDataChampionship['Torneio'] == "BRA2022"
brasilianLeague = dfDataChampionship.loc[filter1]

#mostra dados brasileirao 2020
brasilianLeague.head(3)

In [ ]:
# filtra apenas o Palmeiras
clube = "Palmeiras"
filter2  = (brasilianLeague['Mandante'] == clube) | (brasilianLeague['Visitante'] == clube)
partidas_clube = brasilianLeague.loc[filter2]

# ordena por Data
partidas_clube = partidas_clube.sort_values("Data", ignore_index=True)

# mostra dados jogos Palmeiras
partidas_clube.head(4)

In [ ]:
# cria colunas para jogos Totais
partidas_clube["Vitorias"] = np.nan
partidas_clube["Empates"]  = np.nan
partidas_clube["Derrotas"] = np.nan
partidas_clube["Jogos"]    = np.nan

# cria coluna para jogos como Mandante
partidas_clube["Vit_Mandante"]   = np.nan
partidas_clube["Emp_Mandante"]   = np.nan
partidas_clube["Der_Mandante"]   = np.nan
partidas_clube["Jogos_Mandante"] = np.nan

# cria coluna para Gols como Mandante
partidas_clube["GP_Mandante"] = np.nan
partidas_clube["GC_Mandante"] = np.nan
partidas_clube["SG_Mandante"] = np.nan

# cria coluna para Gols como Visitante
partidas_clube["GP_Visitante"] = np.nan
partidas_clube["GC_Visitante"] = np.nan
partidas_clube["SG_Visitante"] = np.nan


for idx in partidas_clube.index:
    
    # seleciona por indice
    df = partidas_clube.loc[:idx, :]
    
    # conta os totais de Vitorias, Empates e Derrotas
    partidas_clube.loc[idx, "Vitorias"] = (df["Vencedor"] == clube).sum()
    partidas_clube.loc[idx, "Empates"] = (df["Vencedor"] == '-').sum()
    partidas_clube.loc[idx, "Derrotas"] = (df["Derrotado"] == clube).sum()
    partidas_clube.loc[idx, "Jogos"] = len(df.index)
    
    # filtra para jogos que o clube é Mandante
    df_mand = df.loc[df["Mandante"] == clube]
    df_visit = df.loc[df["Visitante"] == clube]
    
    # conta os totais de Vitorias, Empates e Derrotas COMO MANDANTE
    partidas_clube.loc[idx, "Vit_Mandante"] = (df_mand["Vencedor"] == clube).sum()
    partidas_clube.loc[idx, "Emp_Mandante"] = (df_mand["Vencedor"] == '-').sum()
    partidas_clube.loc[idx, "Der_Mandante"] = (df_mand["Derrotado"] == clube).sum()
    partidas_clube.loc[idx, "Jogos_Mandante"] = len(df_mand.index)
    
    # GP, GC e SG como MANDANTE
    partidas_clube.loc[idx, "GP_Mandante"] = df_mand["Mandante_Placar"].sum()
    partidas_clube.loc[idx, "GC_Mandante"] = df_mand["Visitante_Placar"].sum()
    partidas_clube.loc[idx, "SG_Mandante"] = partidas_clube.loc[idx, "GP_Mandante"] - partidas_clube.loc[idx, "GC_Mandante"]

    # GP, GC e SG como VISITANTE
    partidas_clube.loc[idx, "GP_Visitante"] = df_visit["Visitante_Placar"].sum()
    partidas_clube.loc[idx, "GC_Visitante"] = df_visit["Mandante_Placar"].sum()
    partidas_clube.loc[idx, "SG_Visitante"] = partidas_clube.loc[idx, "GP_Visitante"] - partidas_clube.loc[idx, "GC_Visitante"]

# dados como Visitante
partidas_clube["Vit_Visitante"] = partidas_clube["Vitorias"] - partidas_clube["Vit_Mandante"]
partidas_clube["Emp_Visitante"] = partidas_clube["Empates"] - partidas_clube["Emp_Mandante"]
partidas_clube["Der_Visitante"] = partidas_clube["Derrotas"] - partidas_clube["Der_Mandante"]
partidas_clube["Jogos_Visitante"] = partidas_clube["Jogos"] - partidas_clube["Jogos_Mandante"]

# PG_Total, PG_Mandante e PG_Visitante - PG -> Pontos Garantidos
partidas_clube["PG_Total"]     = 3 * partidas_clube["Vitorias"] + partidas_clube["Empates"]
partidas_clube["PG_Mandante"]  = 3 * partidas_clube["Vit_Mandante"] + partidas_clube["Emp_Mandante"]
partidas_clube["PG_Visitante"] = 3 * partidas_clube["Vit_Visitante"] + partidas_clube["Emp_Visitante"]

# cria coluna para Gols Total
partidas_clube["GP_Total"] = partidas_clube["GP_Mandante"] + partidas_clube["GP_Visitante"]
partidas_clube["GC_Total"] = partidas_clube["GC_Mandante"] + partidas_clube["GC_Visitante"]
partidas_clube["SG_Total"] = partidas_clube["GP_Total"] - partidas_clube["GC_Total"]


In [ ]:
partidas_clube.head(5)

In [ ]:
partidas_clube[['ID', 'Torneio', 'Rodada', 'Data', 'Mandante', 'Visitante', 'Arena', 'Placar',
               'GP_Mandante', 'GC_Mandante', 'SG_Mandante', 'GP_Visitante', 'GC_Visitante', 'SG_Visitante',
               'GP_Total', 'GC_Total', 'SG_Total']].tail()